Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f

Добавьте к полученному алгоритму исправления триграммную модель и проверьте, улучшает ли она качество.

In [30]:
from collections import Counter
from nltk import sent_tokenize
from string import punctuation
import gzip
import csv
import pandas as pd
import re
from operator import itemgetter

In [2]:
data = pd.read_csv('lenta-ru-news.csv')

In [3]:
texts = data['text']

In [4]:
punctuation +=  "«»—…“”"
punct = set(punctuation )

In [5]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token
                in tokens_1 if (set(token) - punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token
                in tokens_2 if (set(token) - punct)]
    
    return zip(tokens_1, tokens_2)

In [6]:
def normalize(text):
    normalized = [(word.strip(punctuation)) for word 
                  in text.lower().split()
                  if word.isalpha() 
                  and re.fullmatch(r'[а-я]+', word.strip(punctuation))]
    return [word for word in normalized if word]

In [7]:
corpus = []
for text in texts[:8000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [8]:
vocab = set()

for sent in corpus:
    vocab.update(sent)

In [15]:
words = Counter()

for sent in corpus:
    words.update(sent)

In [18]:
word_freq = dict(words)

for key, value in word_freq.items():
    word_freq[key] = value/len(words)

In [9]:
def n_deletes_helper(word, min_len=0):
    result = set()
    # taking word length into account
    if len(word) > min_len:
        for i in range(len(word)):
            result.add(word[:i] + word[i+1:])
    else:
        result.add(word)
    return result

In [110]:
# нужно сохранять предыдущие удаления!

In [10]:
def n_deletes(word, edit_distance=2, min_len=2):
    forms = {word}
    while edit_distance > 0:
        edit_distance -= 1
        new_forms = set()
        for form in forms:
            new_forms.update(n_deletes_helper(form, min_len))
        forms = new_forms
        
    return forms

In [11]:
#n_deletes('word', edit_distance=2)

In [12]:
sym_vocab = {}
for word in vocab:
    forms = n_deletes(word)
    for form in forms:
        if form not in sym_vocab.keys():
            sym_vocab[form] = word
        else:
            if isinstance(sym_vocab[form], set):
                sym_vocab[form].add(word)
            else:
                sym_vocab[form] = {sym_vocab[form]}
                sym_vocab[form].add(word)

```
There are four different comparison pair types:
dictionary entry==input entry,
delete(dictionary entry,p1)==input entry
dictionary entry==delete(input entry,p2)
delete(dictionary entry,p1)==delete(input entry,p2)
```

In [25]:
for word in sym_vocab['односоронни']:
    print(word_freq[word])

1.141291942478886e-05
2.282583884957772e-05
1.141291942478886e-05
4.565167769915544e-05


In [13]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
good = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [35]:
def most_probable(words, word_freq=word_freq):
    freqs = dict()
    for word in words:
        freqs[word] = word_freq[word]
    return max(freqs.items(), key=itemgetter(1))[0]

In [82]:
def vocab_helper(word, vocab=vocab):
    return word in vocab

In [108]:
'вообще' in vocab

True

In [107]:
sym_vocab['вобще']

KeyError: 'вобще'

In [48]:
def correct(word, sym_vocab=sym_vocab):
    if vocab_helper(word):
        return word
    else:
        if word in sym_vocab.keys(): 
            candidates = sym_vocab[word]
            if isinstance(candidates, set):
                return most_probable(candidates)
            else:
                return candidates
        else:
            # the word is expected to have errors in it,
            # but we don't have a form to replace it with
            return f'*{word}'

In [101]:
def correction_helper(correction, good_prediction_count, bad_prediction_count, error_count, overall):
    append = False
    # if we predicted correctly
    if correction[0] == correction[2]:
        good_prediction_count += 1
    else:
        bad_prediction_count += 1
        append = True
    
    # actual errors
    if correction[0] != correction[1]:
        error_count += 1
    
    # total pairs checked
    overall += 1
    
    return good_prediction_count, bad_prediction_count, error_count, overall, append

In [102]:
cache = dict()
corrections = []
good_preds = 0
bad_preds = 0
errors = 0
total = 0

for i in range(len(good)):
    append = False
    pairs = align_words(good[i], bad[i])
    for pair in pairs:
        left = pair[0]
        right = pair[1]
        if right in cache.keys():
            correction = left, right, cache[right]
        else:
            corrected_word = correct(right)
            correction = left, right, corrected_word
            cache[right] = corrected_word


        good_preds, bad_preds, errors, total, append = correction_helper(correction,
                                                                         good_preds,
                                                                         bad_preds,
                                                                         errors,
                                                                         total)
        if append:
            corrections.append(correction)

    

In [103]:
Counter(corrections).most_common()

[(('вообще', 'вобще', '*вобще'), 18),
 (('вообще', 'ваще', '*ваще'), 17),
 (('естественно', 'естесственно', '*естесственно'), 17),
 (('хочется', 'хочеться', '*хочеться'), 16),
 (('кстати', 'кстате', '*кстате'), 16),
 (('очень', 'ооочень', '*ооочень'), 14),
 (('что-то', 'что-то', '*что-то'), 11),
 (('как-то', 'както', '*както'), 9),
 (('очень', 'оооочень', '*оооочень'), 9),
 (('это', 'ето', 'место'), 9),
 (('что-то', 'чтото', '*чтото'), 7),
 (('периодически', 'переодически', '*переодически'), 7),
 (('получается', 'получаеться', '*получаеться'), 6),
 (('насчет', 'нащет', '*нащет'), 6),
 (('3', '3', '*3'), 6),
 (('здесь', 'сдесь', '*сдесь'), 6),
 (('какой-то', 'какой-то', '*какой-то'), 5),
 (('можно', 'можна', '*можна'), 5),
 (('какие-то', 'какие-то', '*какие-то'), 5),
 (('девчонки', 'девченки', '*девченки'), 5),
 (('что', 'што', 'шторм'), 5),
 (('наконец-то', 'наконецто', '*наконецто'), 5),
 (('очень', 'ооооочень', '*ооооочень'), 5),
 (('из-за', 'изза', 'лиззка'), 5),
 (('кто-то', 'ктото

In [ ]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cached = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cached.get(pair[1], correction(pair[1]))
        cached[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)